In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

In [9]:
def read_and_preprocess(path):
    archive = zipfile.ZipFile(path, 'r')
    file = archive.open("MetroPT3(AirCompressor).csv")
    df = pd.read_csv(file)
    df.drop("Unnamed: 0",axis=1,inplace=True)
    df["Failure"] = 0
    df.timestamp = pd.to_datetime(df.timestamp)
    df = df.set_index("timestamp")
    df.loc["2020-04-18 00:00:00":"2020-04-18 23:59:00","Failure"] = 1
    df.loc["2020-05-29 23:30:00":"2020-05-30 06:00:00","Failure"] = 1
    df.loc["2020-06-05 10:00:00":"2020-06-07 14:30:00","Failure"] = 1
    df.loc["2020-07-15 14:30:00":"2020-07-15 19:00:00","Failure"] = 1
    print("Reading and preprocess is successful!")
    return df

In [10]:
data = read_and_preprocess("data.zip")

Reading and preprocess is successful!


In [11]:
def visualize_timeseries(df,start_time,end_time):

    df_slice = df.loc[start_time:end_time]
    

,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,Failure
timestamp,,,,,,,,,,,,,,,,
2020-02-01 00:00:00,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
2020-02-01 00:00:10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
2020-02-01 00:00:19,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
2020-02-01 00:00:29,-0.012,9.328,9.312,-0.022,9.328,53.425,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
2020-02-01 00:00:39,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
